# Bedrock AgentCore Gateway を使用して OpenAPI API を MCP ツールに変換する

## 概要
Bedrock AgentCore Gateway は、お客様がインフラやホスティングを管理することなく、既存の API を完全に管理された MCP サーバーに変換する方法を提供します。お客様は JSON または YAML 形式の OpenAPI 仕様を持ち込むことができます。OAuth2 で保護されたエンタープライズサポート API を使用するカスタマーサービスエージェントを実演します。

Gateway ワークフローには、エージェントを外部ツールに接続するための以下のステップが含まれます：
* **Gateway 用のツールを作成する** - REST API 用の OpenAPI 仕様などのスキーマを使用してツールを定義します。OpenAPI 仕様は Amazon Bedrock AgentCore によって解析され、Gateway が作成されます。
* **Gateway エンドポイントを作成する** - インバウンド認証を備えた MCP エントリーポイントとして機能する gateway を作成します。
* **Gateway にターゲットを追加する** - gateway がリクエストを特定のツールにルーティングする方法を定義する OpenAPI ターゲットを設定します。OpenAPI ファイルの一部であるすべての API は MCP 互換のツールになり、Gateway エンドポイント URL を通じて利用可能になります。各 OpenAPI Gateway ターゲットに対して OAuth を使用したアウトバウンド認証を設定します。
* **エージェントコードを更新する** - 統合された MCP インターフェースを通じて設定されたすべてのツールにアクセスするために、エージェントを Gateway エンドポイントに接続します。

![仕組み](images/openapis-oauth-gateway.png)

### チュートリアルの詳細


| 情報                 | 詳細                                                     |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ | インタラクティブ                                         |
| AgentCore コンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agent                                             |
| Gateway ターゲットタイプ | OpenAPI                                                   |
| エージェント         | カスタマーサポートエージェント                           |
| インバウンド認証 IdP | Okta                                                      |
| アウトバウンド認証   | OAuth                                                     |
| LLM モデル           | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| チュートリアルコンポーネント | AgentCore Gateway の作成と AgentCore Gateway の呼び出し |
| チュートリアル業種   | 業種横断的                                               |
| 例の複雑さ           | 簡単                                                     |
| 使用 SDK             | boto3                                                     |

チュートリアルの最初の部分では、いくつかの AmazonCore Gateway ターゲットを作成します

### チュートリアルのアーキテクチャ
このチュートリアルでは、OpenAPI yaml/json ファイルで定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。
デモンストレーションの目的で、サポートチケットに関連するクエリに回答するカスタマーサポートエージェントを構築します。エージェントは Zendesk サポート API の OpenAPI を使用します。このソリューションは Amazon Bedrock モデルを使用する Langchain エージェントを使用しています

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+ を搭載した Jupyter notebook
* uv
* AWS 認証情報
* Okta
    - client_id
    - client_secret
    - あなたの Okta ドメイン（例：dev-123456.okta.com）
    - OAuth2 認可サーバー ID（通常は default）

In [ ]:
# 最新の botocore と boto3 ライブラリをダウンロードしていることを確認してください。
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
# SageMaker ノートブックを使用していない場合は AWS 認証情報を設定する
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_PROFILE'] = 'cline2'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
import os
import sys

# 現在のスクリプトのディレクトリを取得する
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック（例：Jupyter）

# utils.py があるディレクトリ（一つ上のレベル）に移動する
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# sys.path に追加する
sys.path.insert(0, utils_dir)

# 今、 utils をインポートできます
import utils

In [ ]:
#### Gateway が引き受ける IAM ロールを作成する
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway へのインバウンド認証のための Okta の設定

以下は Okta OAuth オーソライザーを作成するための手順です -

* [こちら](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/) の指示に従い、クライアント認証情報付与タイプのアプリケーションを作成します。Okta サブスクリプションをお持ちの場合は、管理コンソールにログインして [こちら](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/) に概説されている手順に従ってください。Okta サブスクリプションをお持ちでない場合は、無料トライアルにサインアップする必要があります。
* 管理者 -> アプリケーション -> シークレットを持つクライアントを作成します。トークンリクエストでの所有証明 (DPoP) ヘッダーの要求を無効にします
* Okta 管理者 -> セキュリティ -> API に移動します。デフォルトの認可サーバーを使用し、追加のスコープ（例：InvokeGateway）で変更します。オプションでアクセスポリシーとクレームを追加できます
* カスタムスコープを定義します。認可サーバー名をクリックし、スコープタブに移動して、「スコープを追加」をクリックします
* 設定が完了したら、デフォルトの認可サーバー（別名ディスカバリー URI）のメタデータ URI と、以下に示すように Gateway を作成するためのカスタム JWT オーソライザーを設定するための ClientID/Secret が必要になります

# Okta オーソライザーを使用した受信認証のためのゲートウェイを作成する

In [ ]:
import boto3
from pprint import pprint
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

OKTA_DISCOVERY_URL="https://<YOUR OKTA DOMAIN>/oauth2/<Your app>/.well-known/openid-configuration"
OKTA_AUDIENCE="<Your audience>" # 例えば MCPGateway。Okta の設定と一致する必要があります

auth_config = {
        "customJWTAuthorizer": {
            "allowedAudience": [OKTA_AUDIENCE],
            "discoveryUrl": OKTA_DISCOVERY_URL
        }
}
create_response = gateway_client.create_gateway(name='OpenAPIOktaGwy2',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールは Gateway の作成・一覧表示・取得・削除の権限を持っている必要があります
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway created from sdk with Okta Authorizer'
)
pprint(create_response)
# GatewayTarget 作成に使用される GatewayID を取得する
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]

# Zendesk サポート API を Bedrock AgentCore Gateway を使用して MCP ツールに変換する

### アウトバウンド認証情報プロバイダーの作成

In [ ]:
from botocore.config import Config
ZENDESK_DOMAIN="<Zendek domain url>"
ZENDESK_AUTH_ENDPOINT="https://<Zendeskl-domain>/oauth/authorizations/new"
ZENDESK_TOKEN_ENDPOINT="https://<Zendesk-domain>/oauth/tokens"
ZENDESK_CLIENT_ID="" # あなたの Zendesk OAuth クライアント - クライアント ID
ZENDESK_SECRET=""  # あなたの Zendesk OAuth クライアント - クライアント ID

sdk_config = Config(
    region_name=os.environ['AWS_DEFAULT_REGION'],
    retries={"max_attempts": 2, "mode": "standard"},
)

acps = boto3.client(
    service_name="bedrock-agentcore-control",
    config=sdk_config,
)

provider_config= {
    "customOauth2ProviderConfig": {
         "oauthDiscovery": {
             "authorizationServerMetadata": {
                 "issuer": ZENDESK_DOMAIN,
                 "authorizationEndpoint": ZENDESK_AUTH_ENDPOINT,
                 "tokenEndpoint": ZENDESK_TOKEN_ENDPOINT,
                 "responseTypes": ["token"]
             }
         },
         "clientId": ZENDESK_CLIENT_ID,
         "clientSecret": ZENDESK_SECRET
     }
 }

response = acps.create_oauth2_credential_provider(
    name="ZendeskOAuthTokenCfg", 
    credentialProviderVendor="CustomOauth2", 
    oauth2ProviderConfigInput=provider_config
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress Credentials provider ARN, {credentialProviderARN}")

### OpenAPI ターゲットの作成

#### Zendesk サポート OpenAPI yaml ファイルを S3 にアップロードする

In [ ]:
# S3 クライアントを作成する
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWS アカウント ID とリージョンを取得する
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters

パラメータを定義する
bucket_name = '' # OpenAPI json ファイルをアップロードする s3 バケット。
file_path = 'openapi-specs/Zendesk-support-apis.yaml'
object_key = 'Zendesk-support-apis.yaml'
# ファイルを put_object を使用してアップロードし、レスポンスを読み取る
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # アップロードされたオブジェクトの ARN をアカウント ID とリージョンで構築する
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### ゲートウェイ ターゲットの作成

OpenAPI ファイル内のサーバー URL が、あなた自身のエンドポイント URL を指していることを確認してください。Gateway は OpenAPI ファイルからサーバー URL を読み取り、そのエンドポイントを呼び出します。S3 にアップロードする前に、この変更を必ず行ってください。

In [ ]:
# S3 Uri for OpenAPI 仕様ファイル
openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

credential_config = [
    {
        "credentialProviderType" : "OAUTH",
        "credentialProvider": {
            "oauthCredentialProvider": {
                "providerArn": credentialProviderARN, 
                "scopes": ["tickets:read", "read", "tickets:write", "write"] 
            }
        }
    }
  ]

target_name="DemoOpenAPIGW"
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=target_name,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=openapi_s3_target_config,
    credentialProviderConfigurations=credential_config)

# 不具合を報告するためのリクエスト ID とタイムスタンプを表示します。問題や不具合を報告する際には、これらを含めてください
response_metadata = response['ResponseMetadata']

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装する Bedrock AgentCore Gateway を通じて AWS ツールとシームレスに統合されています。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

その核心部分として、Bedrock AgentCore Gateway は基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠のゲートウェイとして機能します。これらの API により、MCP 準拠のクライアントや SDK は、安全で標準化された方法で利用可能なツールを発見し、対話することができます。Strands エージェントが AWS サービスにアクセスする必要がある場合、これらの MCP 標準化されたエンドポイントを使用してゲートウェイと通信します。

ゲートウェイの実装は (MCP 認証仕様)[https://modelcontextprotocol.org/specification/draft/basic/authorization] に厳密に準拠しており、堅牢なセキュリティとアクセス制御を確保しています。これは、Strands エージェントによるすべてのツール呼び出しが認証ステップを経由することを意味し、強力な機能を提供しながらセキュリティを維持します。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを発見し、次に InvokeTools を使用して特定のアクションを実行します。ゲートウェイはすべての必要なセキュリティ検証、プロトコル変換、サービス連携を処理し、プロセス全体をシームレスかつ安全にします。

このアーキテクチャアプローチにより、MCP 仕様を実装するあらゆるクライアントや SDK がゲートウェイを通じて AWS サービスと対話できるようになり、AI エージェント統合のための汎用性と将来性のあるソリューションとなっています。

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# Okta からインバウンド認証用のアクセストークンをリクエストする

In [ ]:
print("Requesting the access token from Okta authorizer")
import requests
from requests.auth import HTTPBasicAuth

# 実際の値に置き換えてください
OKTA_DOMAIN = "Your Okta domain URL"
AUTH_SERVER_ID = "Okta app id"
CLIENT_ID = "<Okta client credentials client id>"
CLIENT_SECRET = "<Okta client credentials secret>"

TOKEN_URL = f"{OKTA_DOMAIN}/oauth2/{AUTH_SERVER_ID}/v1/token"

response = requests.post(
    TOKEN_URL,
    auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET),
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"grant_type": "client_credentials", "scope": "InvokeGateway"}
)

if response.status_code == 200:
    token = response.json()["access_token"]
    print("Access Token:", token)
else:
    print("Failed to get token:", response.status_code, response.text)

# Zendesk サポート API を使用した Bedrock AgentCore Gateway によるカスタマーサポートエージェントへの問い合わせ

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials に設定された IAM グループ/ユーザーは Bedrock モデルへのアクセス権を持っている必要があります
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# ルートの strands ロガーを設定します。問題をデバッグしている場合は、DEBUG に変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを確認するためのハンドラーを追加する
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # モデルとツールを使用したエージェントの作成
    agent = Agent(model=yourmodel,tools=tools) ## あなたは任意のモデルに置き換えることができます
    ```python
#print(f"エージェントに読み込まれたツールは {agent.tool_names} です")
```
    ```python
#print(f"エージェント内のツール設定は {agent.tool_config} です")
```
    
    # エージェントをサンプルプロンプトで呼び出します。これは MCP listTools のみを呼び出し、LLM がアクセスできるツールのリストを取得します。以下は実際にはどのツールも呼び出しません。
    #agent("こんにちは、あなたが利用できるすべてのツールをリストアップしてもらえますか")
    agent("Count the number of support tickets")
    # MCP ツールを明示的に呼び出します。MCP ツール名と引数は、AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります
    result = client.call_tool_sync(
    tool_use_id="count-tickets-1", # ログを確認するためのハンドラーを追加する0
    name="DemoOpenAPIGW___CountTickets", # ログを確認するためのハンドラーを追加する1
    )
    # ログを確認するためのハンドラーを追加する2
    print(f"Tool Call result: {result['content'][0]['text']}")


# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されており、クリーンアップの一環として手動で削除する必要があるかもしれません。これは実行する例によって異なります。

## ゲートウェイの削除 (オプション)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)